In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df = pd.read_csv('/kaggle/input/mustard/mustard_text.csv')

In [5]:
df.head()

,SCENE,KEY,SENTENCE,END_TIME,SPEAKER,SHOW,Sarcasm,Sarcasm_Type,Implicit_Emotion,Explicit_Emotion,Valence,Arousal
0,1_10004,1_10004_c_00,"Well, I'm sure that, uh, you...\nhave a lot of...",0:06,PERSON,BBT,NaN,NaN,NaN,NaN,NaN,NaN
1,1_10004,1_10004_c_01,Who was he?,0:08,SHELDON,BBT,NaN,NaN,NaN,NaN,NaN,NaN
2,1_10004,1_10004_c_02,His name is Ron.\nI met him at my prayer group.,0:12,PERSON,BBT,NaN,NaN,NaN,NaN,NaN,NaN
3,1_10004,1_10004_c_03,How long have you been involved with him?,0:14,SHELDON,BBT,NaN,NaN,NaN,NaN,NaN,NaN
4,1_10004,1_10004_c_04,A few months.,0:16,PERSON,BBT,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.isnull().sum()

SCENE                  0
KEY                    0
SENTENCE               0
END_TIME               0
SPEAKER                0
SHOW                   0
Sarcasm             4839
Sarcasm_Type        4839
Implicit_Emotion    4839
Explicit_Emotion    4839
Valence             4839
Arousal             4839
dtype: int64

In [7]:
dff = df.dropna()

In [8]:
dff.head()

,SCENE,KEY,SENTENCE,END_TIME,SPEAKER,SHOW,Sarcasm,Sarcasm_Type,Implicit_Emotion,Explicit_Emotion,Valence,Arousal
5,1_10004,1_10004_u,"And of those few months, how long have you bee...",0:07,SHELDON,BBT,0.0,NONE,Anger,Anger,2.0,7.0
14,1_10009,1_10009_u,"Let the dead man talk. So, why do you think that?",0:05,PENNY,BBT,0.0,NONE,Happiness,Happiness,6.0,5.0
18,1_1001,1_1001_u,"What else? Sell it on eBay as ""slightly used.""",0:04,RAJ,BBT,0.0,NONE,Excitement,Excitement,6.0,5.0
24,1_1003,1_1003_u,"Good idea, sit with her. Hold her, comfort her...",0:08,HOWARD,BBT,1.0,PRO,Anger,Sadness,3.0,6.0
31,1_10190,1_10190_u,"Well, now that I've given up string theory, I'...",0:11,SHELDON,BBT,0.0,NONE,Neutral,Neutral,5.0,5.0


In [9]:
dff.value_counts

<bound method DataFrame.value_counts of              SCENE             KEY  \
5          1_10004       1_10004_u   
14         1_10009       1_10009_u   
18          1_1001        1_1001_u   
24          1_1003        1_1003_u   
31         1_10190       1_10190_u   
...            ...             ...   
6014  3_S06E02_398  3_S06E02_398_u   
6021  3_S06E03_366  3_S06E03_366_u   
6030  3_S06E05_355  3_S06E05_355_u   
6036  3_S06E06_143  3_S06E06_143_u   
6040  3_S06E07_272  3_S06E07_272_u   

                                               SENTENCE     END_TIME  \
5     And of those few months, how long have you bee...         0:07   
14    Let the dead man talk. So, why do you think that?         0:05   
18       What else? Sell it on eBay as "slightly used."         0:04   
24    Good idea, sit with her. Hold her, comfort her...         0:08   
31    Well, now that I've given up string theory, I'...         0:11   
...                                                 ...          ...   

In [10]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1202 entries, 5 to 6040
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   SCENE             1202 non-null   object 
 1   KEY               1202 non-null   object 
 2   SENTENCE          1202 non-null   object 
 3   END_TIME          1202 non-null   object 
 4   SPEAKER           1202 non-null   object 
 5   SHOW              1202 non-null   object 
 6   Sarcasm           1202 non-null   float64
 7   Sarcasm_Type      1202 non-null   object 
 8   Implicit_Emotion  1202 non-null   object 
 9   Explicit_Emotion  1202 non-null   object 
 10  Valence           1202 non-null   float64
 11  Arousal           1202 non-null   float64
dtypes: float64(3), object(9)
memory usage: 122.1+ KB


In [11]:
dff['Sarcasm_Type'].value_counts()

Sarcasm_Type
NONE    601
PRO     333
ILL     178
EMB      87
LIK       3
Name: count, dtype: int64

In [12]:
labels = np.array(dff['Sarcasm'])

In [13]:
from transformers import BertModel, BertTokenizer
import torch

In [14]:
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [19]:
# Define a function to extract features for each transaction
def extract_features(SENTENCE):
    # Tokenize the text
    input_ids = torch.tensor([tokenizer.encode(SENTENCE, add_special_tokens=True)])
    # Get the hidden states for each token
    with torch.no_grad():
        outputs = model(input_ids)
        hidden_states = outputs[2]
    # Concatenate the last 4 hidden states
    token_vecs = []
    for layer in range(-4, 0):
        token_vecs.append(hidden_states[layer][0])
    # Calculate the mean of the last 4 hidden states
    features = []
    for token in token_vecs:
        features.append(torch.mean(token, dim=0))
    # Return the features as a tensor
    return torch.stack(features)

In [20]:
# Extract features for each transaction
features = []
for i in range(len(dff)):
    features.append(extract_features(dff.iloc[i]["SENTENCE"]))
# Concatenate the features and convert to a numpy array
features = torch.cat(features).numpy()

In [21]:
features

array([[ 0.23776594, -0.17895852, -0.00784114, ..., -0.4028634 ,
         0.2962938 , -0.15918873],
       [ 0.60434735, -0.15395905,  0.13442115, ..., -0.49247217,
         0.16818556, -0.21582215],
       [ 0.57802284, -0.05877127,  0.06738599, ..., -0.60506487,
         0.14738731, -0.09102236],
       ...,
       [ 0.15257971,  0.13167292, -0.5081716 , ..., -0.13758367,
         0.22323285, -0.40706176],
       [ 0.19974917,  0.20950432, -0.46279594, ..., -0.2839372 ,
         0.07469081, -0.33474264],
       [ 0.3134834 ,  0.15238431, -0.38852817, ..., -0.2188434 ,
         0.08912823, -0.27032825]], dtype=float32)

In [27]:
# Saving the features as a .npy file
bert = np.save("news_features.npy", features)

In [28]:
sarcasm_type = dff['Sarcasm_Type'].values
sarcasm_type

array(['NONE', 'NONE', 'NONE', ..., 'PRO', 'ILL', 'ILL'], dtype=object)

In [29]:
print(features.shape)
print(sarcasm_type.shape)

(4808, 768)
(1202,)


In [30]:
print(labels.shape)

(1202,)


In [40]:
lab = dff['Sarcasm'].values
lab

array([0., 0., 0., ..., 1., 1., 1.])

In [41]:
labels

array([0., 0., 0., ..., 1., 1., 1.])

In [42]:
from sklearn.model_selection import train_test_split

In [ ]:
#X_train_val_text, X_test_text, y_train_val, y_test = train_test_split(bert, lab, random_state=104, test_size=0.2)
#X_train_text, X_val_text, y_train, y_val = train_test_split(X_train_val_text,y_train_val, random_state=104, test_size=0.1)

In [35]:
'''import tensorflow as tf
import tensorflow_hub as hub
# import tensorflow_text
from tensorflow import keras
from tensorflow.keras import models
import pandas as pd
import numpy as np
import sklearn
import keras.backend as K'''

2024-05-27 05:20:05.440814: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-27 05:20:05.441039: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-27 05:20:05.602065: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [36]:
'''m = tf.keras.layers.Dense(512, activation='relu', name="text_output1")(m)
m = tf.keras.layers.Dropout(0.2, name="text_dropout1")(m)
m = tf.keras.layers.Dense(256, activation='relu', name="text_output2")(m)
m = tf.keras.layers.Dropout(0.2, name="text_dropout3")(m)
out_txt = tf.keras.Model(inputs=[text_input], outputs = m)'''

NameError: name 'm' is not defined

In [45]:
features_reshaped = features.reshape((1202, -1))
dataset = np.hstack((features_reshaped, labels.reshape((-1, 1))))

In [46]:
features_reshaped.shape

(1202, 3072)

In [47]:
dataset

array([[ 2.37765938e-01, -1.78958520e-01, -7.84114003e-03, ...,
         1.34386897e-01, -1.29903227e-01,  0.00000000e+00],
       [-4.98979278e-02,  7.38104358e-02, -1.08013518e-01, ...,
         4.12124813e-01,  3.78343277e-04,  0.00000000e+00],
       [ 2.37545565e-01, -1.26816317e-01, -3.00965846e-01, ...,
         1.39245138e-01, -2.16567710e-01,  0.00000000e+00],
       ...,
       [ 1.12979785e-01,  4.06526148e-01, -2.07852889e-02, ...,
        -2.02783406e-01,  2.27192849e-01,  1.00000000e+00],
       [-1.88128397e-01, -1.90799162e-01,  5.75169921e-03, ...,
         4.48750257e-02,  2.35968590e-01,  1.00000000e+00],
       [ 2.27860391e-01, -3.46487798e-02, -4.73305434e-01, ...,
         8.91282260e-02, -2.70328254e-01,  1.00000000e+00]])

In [48]:
dataset.shape

(1202, 3073)

In [49]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

X_train, y_train = train_data[:, :-1], train_data[:, -1]
X_test, y_test = test_data[:, :-1], test_data[:, -1]

## Logistic Regression

In [59]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [60]:
# Evaluate the classifier on the testing set
score = lr.score(X_test, y_test)
print("Accuracy:", score)

Accuracy: 0.6058091286307054


In [61]:
# Predict the labels of the testing set
y_pred = lr.predict(X_test)

In [62]:
from sklearn.metrics import confusion_matrix, classification_report

In [63]:
cm = confusion_matrix(y_test, y_pred)
cr = classification_report(y_test, y_pred)

In [64]:
print("Confusion Matrix:\n", cm)

Confusion Matrix:
 [[72 51]
 [44 74]]


In [65]:
print('Classification Report\n', cr)

Classification Report
               precision    recall  f1-score   support

         0.0       0.62      0.59      0.60       123
         1.0       0.59      0.63      0.61       118

    accuracy                           0.61       241
   macro avg       0.61      0.61      0.61       241
weighted avg       0.61      0.61      0.61       241

